In [118]:
import sys
import os
import pandas as pd
from sqlalchemy import create_engine
from warnings import filterwarnings
import csv
import re, os, random, string, codecs
import sys
from collections import Counter, defaultdict
import time
import tqdm
project_home = os.environ['PACKAGE_HOME']
from Development.helpers import general_helpers

In [119]:
import configparser

config = configparser.ConfigParser()
config.read(project_home + '/Development/config.ini')

db_con = general_helpers.connect_to_db(config['DATABASE']['HOST'], config['DATABASE']['USERNAME'],
                                       config['DATABASE']['PASSWORD'], config['DATABASE']['NEW_DB'])

db_folder = 'upload_20191231'
disambig_folder = "{}{}/disambig_output".format(config['FOLDERS']['WORKING_FOLDER'], db_folder)


print(disambig_folder)
old_db = config['DATABASE']['OLD_DB']
new_db = config['DATABASE']['NEW_DB']
new_db_timestamp = new_db.replace('patent_', '')
print(old_db, "|", new_db, "|" , new_db_timestamp)

/pipeline_datadir/upload_20191231/disambig_output
patent_20191008 | patent_20191231 | 20191231


In [127]:
# 1. create output file for wide format

# get disambig cols needed
result = db_con.execute("select column_name from information_schema.columns where table_schema = '{0}' and table_name = '{1}'".format(new_db, 'persistent_inventor_disambig'))
pid_cols = [r[0] for r in result]
disambig_cols = [x for x in pid_cols if x.startswith('disamb')]
print(disambig_cols)
print('######################################################################\n')

outfile_name = disambig_folder +'/persistent_inventor_wide.tsv'

# TODO: add back in the + part below for final script - ignoring for now since persistent_inventor_disambig has that column
header = ['current_rawinventor_id', 'old_rawinventor_id'] + disambig_cols #+ ['disamb_inventor_id' + new_db_timestamp]
print(header)
header_df = pd.DataFrame(columns = header)
header_df.to_csv(outfile_name, index=False, header=True, sep='\t')

['disamb_inventor_id_20171226', 'disamb_inventor_id_20171003', 'disamb_inventor_id_20170808', 'disamb_inventor_id_20180528', 'disamb_inventor_id_20181127', 'disamb_inventor_id_20190312', 'disamb_inventor_id_20190820', 'disamb_inventor_id_20191008', 'disamb_inventor_id_20191231']
######################################################################

['current_rawinventor_id', 'old_rawinventor_id', 'disamb_inventor_id_20171226', 'disamb_inventor_id_20171003', 'disamb_inventor_id_20170808', 'disamb_inventor_id_20180528', 'disamb_inventor_id_20181127', 'disamb_inventor_id_20190312', 'disamb_inventor_id_20190820', 'disamb_inventor_id_20191008', 'disamb_inventor_id_20191231']


In [99]:
outfile_name_newinventors = disambig_folder +'/persistent_inventor_long_20191231.tsv'
header = ['uuid', 'database_update', 'inventor_id']
header_df = pd.DataFrame(columns = header)
header_df.to_csv(outfile_name, index=False, header=True, sep='\t')

In [3]:
# get total rows in the rawinventor table
result = db_con.execute('select count(*) from {}.{}'.format(new_db,'rawinventor'))
total_rows = [r[0] for r in result][0]
total_rows

17428664

In [103]:
# Syntax testing
limit = 5 
offset = 0
sql_stmt_template = "select uuid, inventor_id from {0}.{1} order by uuid limit {2} offset {3}".format(new_db, 'rawinventor', limit, offset)
print(sql_stmt_template)
result = db_con.execute(sql_stmt_template)
# row needs to be uuid, database, inventor id
chunk_results = [ (r[0], new_db_timestamp, r[1]) for r in result]
print(chunk_results)

chunk_df = pd.DataFrame(chunk_results, columns = pid_long_cols)
print(chunk_df.head())

# for pair in chunk_results:
#     uuid = pair[0]
#     inventor_id = pair[1]

#     insert_row = (uuid, new_db_timestamp, inventor_id)
#     insert_stmt_template = "insert into {0}.{1} (uuid, database_update, inventor_id) values {2}".format(new_db,'persistent_inventor_disambig_long',insert_row)
#     print(insert_stmt_template)

select uuid, inventor_id from patent_20191231.rawinventor order by uuid limit 5 offset 0
[('0000hccb98m2kc6g1v7128k5w', '20191231', '4341225-2'), ('0000kwt5abwdu9f4av6zoa61t', '20191231', '4339721-2'), ('0000n6xqianutadbzbgzwled7', '20191231', '6610738-6'), ('0000n8nqsxhrztn7djlxou00k', '20191231', '6448562-2'), ('0000p6jf5l8yzv04wimaoabab', '20191231', '4127345-1')]
                        uuid database_update inventor_id
0  0000hccb98m2kc6g1v7128k5w        20191231   4341225-2
1  0000kwt5abwdu9f4av6zoa61t        20191231   4339721-2
2  0000n6xqianutadbzbgzwled7        20191231   6610738-6
3  0000n8nqsxhrztn7djlxou00k        20191231   6448562-2
4  0000p6jf5l8yzv04wimaoabab        20191231   4127345-1


In [ ]:
# 2. rawinventor rows from current db update: insert into persistent_inventor_disambig_long table in chunks
limit = 300000
offset = 0

start = time.time()
itr = 0

print('Estimated # of rounds: ', total_rows/300000)

while True:
    print('###########################################\n')
    print('Next iteration... ', itr)
    
    sql_stmt_template = "select uuid, inventor_id from {0}.{1} order by uuid limit {2} offset {3};".format(new_db, 'rawinventor', limit, offset)
    print(sql_stmt_template)
    result = db_con.execute(sql_stmt_template)
    
    # r = tuples of (uuid, inventor_id)
    chunk_results = [ (r[0], new_db_timestamp, r[1]) for r in result]

    # means we have no more result batches to process! done
    if len(chunk_results) == 0:
        break
        
    chunk_df = pd.DataFrame(chunk_results, columns = pid_long_cols)
    chunk_df.to_csv(outfile_name_newinventors, index=False, header=False, mode = 'a', sep='\t')
    
    # test 
    if offset == 600000:
        break

         
    # continue
    offset+=limit 
    itr+=1

    if itr == 1:
        print('Time for 1 iteration: ', time.time() - start, ' seconds')
    print('###########################################\n')

    
print('###########################################')
print('total time taken:', round(time.time() - start, 2), ' seconds')
print('###########################################')


In [129]:
# 3. get column names of persistent_inventor_disambig_long table for pivoting long -> wide
result = db_con.execute("select column_name from information_schema.columns where table_schema = '{0}' and table_name = '{1}'".format(new_db, 'persistent_inventor_disambig_long'))
result_cols = [r[0] for r in result]
pid_long_cols = [x for x in result_cols if x != 'id']
pid_long_cols = ['current_uuid', 'old_uuid', 'database_update', 'inventor_id']
print(pid_long_cols)

['current_uuid', 'old_uuid', 'database_update', 'inventor_id']


In [126]:
# Syntax testing
limit = 5
offset = 17000000
sql_stmt_inner = "( select uuid from {0}.{1} order by uuid limit {2} offset {3}) ri".format(new_db, 'rawinventor',  limit, offset)
sql_stmt_template = "select lf.uuid as current_uuid, ri_old.uuid as old_uuid, lf.database_update, lf.inventor_id from {0} left join {1}.{2} lf on ri.uuid = lf.uuid left join {3}.{4} ri_old on lf.uuid = ri_old.uuid;".format(sql_stmt_inner,new_db,'persistent_inventor_disambig_long', old_db, 'rawinventor')

print(sql_stmt_template)
result = db_con.execute(sql_stmt_template)

chunk_results = [r for r in result]
chunk_results

select lf.uuid as current_uuid, ri_old.uuid as old_uuid, lf.database_update, lf.inventor_id from ( select uuid from patent_20191231.rawinventor order by uuid limit 5 offset 17000000) ri left join patent_20191231.persistent_inventor_disambig_long lf on ri.uuid = lf.uuid left join patent_20191008.rawinventor ri_old on lf.uuid = ri_old.uuid;


[('z41yt85v6br6x22t4tgz0j4ej', 'z41yt85v6br6x22t4tgz0j4ej', '20181127', '10015597-1'),
 ('z41yt85v6br6x22t4tgz0j4ej', 'z41yt85v6br6x22t4tgz0j4ej', '20190312', '6624768-2'),
 ('z41yt85v6br6x22t4tgz0j4ej', 'z41yt85v6br6x22t4tgz0j4ej', '20190820', '6624768-2'),
 ('z41yt85v6br6x22t4tgz0j4ej', 'z41yt85v6br6x22t4tgz0j4ej', '20191008', '6624768-2'),
 ('z41yt85v6br6x22t4tgz0j4ej', 'z41yt85v6br6x22t4tgz0j4ej', '20191231', '6624768-2'),
 ('z41yvs6q9vhkv9s91b160yjh6', 'z41yvs6q9vhkv9s91b160yjh6', '20170808', '4980409-3'),
 ('z41yvs6q9vhkv9s91b160yjh6', 'z41yvs6q9vhkv9s91b160yjh6', '20171003', '4980409-3'),
 ('z41yvs6q9vhkv9s91b160yjh6', 'z41yvs6q9vhkv9s91b160yjh6', '20171226', '4980409-3'),
 ('z41yvs6q9vhkv9s91b160yjh6', 'z41yvs6q9vhkv9s91b160yjh6', '20180528', '4980409-3'),
 ('z41yvs6q9vhkv9s91b160yjh6', 'z41yvs6q9vhkv9s91b160yjh6', '20181127', '4980409-3'),
 ('z41yvs6q9vhkv9s91b160yjh6', 'z41yvs6q9vhkv9s91b160yjh6', '20190312', '4980409-3'),
 ('z41yvs6q9vhkv9s91b160yjh6', 'z41yvs6q9vhkv9s91b160

In [140]:
# Syntax testing
chunk_df = pd.DataFrame(chunk_results, columns = pid_long_cols)
chunk_df['database_update'] = 'disamb_inventor_id_' + chunk_df['database_update']

# pre pivot check
print(chunk_df.head())
print('######################################################################\n')

# reset index to get back uuid as column, rename axis to get rid of database_update axis value
chunk_df = chunk_df.pivot(index=['current_uuid','old_uuid'], columns='database_update', values='inventor_id')#.reset_index().rename_axis(None,1)
chunk_df
# post pivot & processing check
#print(chunk_df.head())    
#print('######################################################################\n')

# sort = False will preserve col order
# formatted_chunk_df = pd.concat([header_df, chunk_df], sort=False)

# # replace NAs with empty strings
# formatted_chunk_df.to_csv(outfile_name, index=False, header=False, mode = 'a', sep='\t', na_rep = None)

# formatted_chunk_df

                current_uuid                   old_uuid  \
0  z41yt85v6br6x22t4tgz0j4ej  z41yt85v6br6x22t4tgz0j4ej   
1  z41yt85v6br6x22t4tgz0j4ej  z41yt85v6br6x22t4tgz0j4ej   
2  z41yt85v6br6x22t4tgz0j4ej  z41yt85v6br6x22t4tgz0j4ej   
3  z41yt85v6br6x22t4tgz0j4ej  z41yt85v6br6x22t4tgz0j4ej   
4  z41yt85v6br6x22t4tgz0j4ej  z41yt85v6br6x22t4tgz0j4ej   

               database_update inventor_id  
0  disamb_inventor_id_20181127  10015597-1  
1  disamb_inventor_id_20190312   6624768-2  
2  disamb_inventor_id_20190820   6624768-2  
3  disamb_inventor_id_20191008   6624768-2  
4  disamb_inventor_id_20191231   6624768-2  
######################################################################



ValueError: Length of passed values is 41, index implies 2

'/pipeline_datadir/upload_20191231/disambig_output/persistent_inventor_wide.tsv'

In [115]:
# Syntax testing
# test = pd.concat([chunk_df_p1, chunk_df])
# test.shape # should have 10 rows

# if this works, then write to output file as append - should put in same way?
# concat to header each time for organizing ---- then append to file 

#print(test)

# test[test.isna()] = ''
# test
#print(header_df)
test2 = pd.concat([header_df, chunk_df], sort=False)

#test2

#test3 = 
#header_df
print(test2)

test2.to_csv(outfile_name, index=False, header=False, mode = 'a', sep='\t', na_rep = None)
    



                        uuid disamb_inventor_id_20171226  \
0  0000hccb98m2kc6g1v7128k5w                         NaN   
1  0000kwt5abwdu9f4av6zoa61t                         NaN   
2  0000n6xqianutadbzbgzwled7                         NaN   
3  0000n8nqsxhrztn7djlxou00k                         NaN   
4  0000p6jf5l8yzv04wimaoabab                         NaN   

  disamb_inventor_id_20171003 disamb_inventor_id_20170808  \
0                         NaN                         NaN   
1                         NaN                         NaN   
2                         NaN                         NaN   
3                         NaN                         NaN   
4                         NaN                         NaN   

  disamb_inventor_id_20180528 disamb_inventor_id_20181127  \
0                         NaN                         NaN   
1                         NaN                         NaN   
2                         NaN                         NaN   
3                         Na

In [ ]:
# 4. Need to convert long table into wide and output .tsv - grabbing all uuid rows together for the set of uuids
limit = 300000
offset = 0

start = time.time()
itr = 0

print('Estimated # of rounds: ', total_rows/300000)

while True:
    print('###########################################\n')
    print('Next iteration... ', itr)
    
    sql_stmt_inner = "( select uuid from {0}.{1} order by uuid limit {2} offset {3}) ri".format(new_db, 'rawinventor',  limit, offset)
    sql_stmt_template = "select lf.uuid, lf.database_update, lf.inventor_id from {0} inner join {1}.{2} lf on ri.uuid = lf.uuid;".format(sql_stmt_inner,new_db,'persistent_inventor_disambig_long')

    print(sql_stmt_template)
    result = db_con.execute(sql_stmt_template)

    chunk_results = [r for r in result]
    print(len(chunk_results))
    
    # means we have no more result batches to process! done
    if len(chunk_results) == 0:
        break

    chunk_df = pd.DataFrame(chunk_results, columns = pid_long_cols)
    chunk_df['database_update'] = 'disamb_inventor_id_' + chunk_df['database_update']
    
    # pre pivot check
    print(chunk_df.head())
    print('######################################################################\n')
    
    # reset index to get back uuid as column, rename axis to get rid of database_update axis value
    chunk_df = chunk_df.pivot(index='uuid', columns='database_update', values='inventor_id').reset_index().rename_axis(None,1)
    
    # post pivot & processing check
    print(chunk_df.head())    
    print('######################################################################\n')

    # sort = False will preserve col order
    formatted_chunk_df = pd.concat([header_df, chunk_df], sort=False)
    
    # replace NAs with empty strings
    formatted_chunk_df.to_csv(outfile_name, index=False, header=False, mode = 'a', sep='\t', na_rep = None)
    
    offset+=limit 
    itr+=1

    if itr == 1:
        print('Time for 1 iteration: ', time.time() - start, ' seconds')
    print('###########################################\n')
    
    
print('###########################################')
print('total time taken:', round(time.time() - start, 2), ' seconds')
print('###########################################')

In [ ]:
# 4. load wide table 
#LOAD DATA INFILE 'persistent_inventor_long.tsv' INTO TABLE persistent_inventor_wide FIELDS TERMINATED BY '\t' (uuid, database_update, inventor_id) IGNORE 1 lines;